In [ ]:
cd /content/drive/MyDrive/author_style

/content/drive/MyDrive/author_style


In [ ]:
!pip install SentencePiece
!pip install transformers

from author_style.utils import *

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

from transformers import CamembertConfig
from transformers import TFCamembertForMultipleChoice
from transformers import TFCamembertModel
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

In [ ]:
df_paragraphs, df_sentences = csv_to_dataframes()

In [ ]:
df_paragraphs['text_len'] = df_paragraphs.text.apply(lambda x: len(x))

In [ ]:
pd.options.display.max_rows = 200
pd.options.display.max_seq_items = None
pd.options.display.max_colwidth = 4000
df_paragraphs.sort_values(by='text_len')

# max len : 2566 char ; 2ème : 1468, 1446, 1355, etc.
# min len : 403

,text,author,title,book_date,text_len
11457,- L'Arcille! L'Arcille! Et votre boutique alors! La consigne! Où que vous êtes? Et l'appel? L'appel!,CELINE Louis-Ferdinand,Casse-pipe,1949,100
11149,"– Voyez-vous, je pensais qu'il fallait, pour faire ce travail-là, certaines qualités indispensables.",DURAS Marguerite,Le Square,1955,100
14401,— J’ai mes fiches sur tous nos candidats. L’Académie est un club. Sachons à qui nous avons affaire !,GUTH Paul,Discours de déception à l_Académie française,1987,100
7098,"Je n'insiste pas, je pars. Mais je dis ce qui est éprouvé par tous, même si on ne sait pas le vivre.",DURAS Marguerite,Écrire,1993,100
11456,"- T'as qu'à foncer chez Le Croach, l'en a toujours dans son puce... Y ronfle dessus, le gros cochon!",CELINE Louis-Ferdinand,Casse-pipe,1949,100
...,...,...,...,...,...
12362,"Solennels parmi les couples sans amour, ils dansaient, d’eux seuls préoccupés, goûtaient l’un à l’autre, soigneux, profonds, perdus. Béate d’être tenue et guidée, elle ignorait le monde, écoutait le bonheur dans ses veines, parfois s’admirant dans les hautes glaces des murs, élégante, émouvante, exceptionnelle, femme aimée, parfois reculant la tête pour mieux le voir qui lui murmurait des merveilles point toujours comprises, car elle le regardait trop, mais toujours de toute âme approuvées, qui lui murmurait qu’ils étaient amoureux, et elle avait alors un impalpable rire tremblé, voilà, oui, c’était cela, amoureux, et il lui murmurait qu’il se mourait de baiser et bénir les longs cils recourbés, mais non pas ici, plus tard, lorsqu’ils seraient seuls, et alors elle murmurait qu’ils avaient toute la vie, et soudain elle avait peur de lui avoir déplu, trop sûre d’elle, mais non, ô bonheur, il lui souriait et contre lui la gardait et murmurait que tous les soirs, oui, tous les soirs ils se verraient. Secoué dans son wagon-lit, il se reprochait d’avoir été une brute, une brute de l’avoir traitée de méchante. Après tout, si elle n’avait pas de sympathie pour le boss, elle n’y pouvait rien, pas de sa faute quoi. Elle avait du bon, allez. L’autre jour, chez le tailleur, elle l’avait si gentiment aidé à choisir le tissu, elle s’y était vrai ment intéressée. Elle dormait sûrement en ce moment, si mignonne quand elle dormait. Dors bien, ma chérie, lui dit-il en sa couche remuante, et il lui sourit, ferma les yeux pour dormir avec elle. L’orchestre tzigane stoppa, et ils s’arrêtèrent sans se détacher tandis que les ordinaires, aussitôt séparés, battaient des mains, battaient en vain. Mais sur un regard de Solal, Imre, le premier violon marqué de variole, cligna un sourire complice, essuya ses sueurs et attaqua avec grandeur tandis que les deux étranges, observés par les assis, repartaient en gravité d’amour, bientôt suivis par Imre fioriturant à grands effets de manches flottantes et tenant entre ses dents le billet de banque donné par Solal. Traînant derrière elle des serpentins lancés, lentes algues de toutes couleurs, elle détachait parfois sa main pour rajuster sa coiffure et n’y parvenait pas, oh tant pis, et puis son nez brillait peut-être, oh tant pis puisqu’elle était sa belle, puisqu’il le lui disait. La belle du seigneur, se disait-elle, souriant aux anges. Mais il ne parvenait pas à s’endormir et il se demandait si elle avait pensé à fermer le compteur du gaz. L’embêtant, c’était qu’elle allait être toute seule dans la villa, rien qu’avec une femme de ménage le matin, puisque Mariette ne reprendrait son service que dans un mois à peu près, et il n’y avait pas que le compteur du gaz, il y avait les verrous de la porte d’entrée qu’elle oublierait sûrement de pousser le soir avant de se coucher, et puis ses vitamines du matin, elle ne penserait sûrement pas à les prendre, ah que de soucis. Joue contre joue, elle et lui, secrets, lentement virant. Ô elle, murmurait-il, elle, tous les charmes, alpiniste de l’Himalaya en béret écossais, reine des bêtes de porcelaine, ô son sourire de demeurée lorsqu’elle était seule,

In [ ]:
X = df_paragraphs.text.tolist()
y = df_paragraphs.author.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y)
LabelEncoder()
print(le.classes_)
y = le.transform(y)

['BASTIDE Francois-Regis' 'BAZIN Herve' 'BAZIN Hervé' 'CAMUS Albert'
 'CELINE Louis-Ferdinand' 'COHEN Albert' 'DEBEAUVOIR SIMONE'
 'DEBEAUVOIR Simone' 'DURAS Marguerite' 'ECHENOZ Jean' 'GARY Romain'
 'GASCAR Pierre' 'GIONO Jean' 'GRAINVILLE Patrick' 'GUTH Paul'
 'JOFFO Joseph' 'KESSEL Joseph' 'MODIANO Patrick' 'PEREC Georges'
 'QUEFFELEC Henri' 'QUEFFELEC Yann' 'SAGAN Fraçoise' 'SARRAUTE Nathalie'
 'SCHOENDOERFFER Pierre' 'VIAN Boris' 'YOURCENAR Marguerite']


In [ ]:
y_cat = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3)

In [ ]:
LONGUEUR_MAX_PARAGRAPH = 512

#instancier le tokenizeur
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-camembert-base')

#préparation des données d'entrainement/test
output_tokenizer_train = tokenizer(X_train, max_length=LONGUEUR_MAX_PARAGRAPH, padding="max_length", truncation=True, return_tensors='tf', add_special_tokens=True)
output_tokenizer_test = tokenizer(X_test, max_length=LONGUEUR_MAX_PARAGRAPH, padding="max_length", truncation=True, return_tensors='tf', add_special_tokens=True)

In [ ]:
#Instanciation du modèle
transformer_model = TFCamembertModel.from_pretrained('jplu/tf-camembert-base')

#Définition des formats d'entrée
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_PARAGRAPH,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_PARAGRAPH,), name='masked_token', dtype='int32')

#Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids, entrees_masks])
#print(sortie_camemBERT.shape)
print(sortie_camemBERT)
#>>TFBaseModelOutputWithPooling(
    #last_hidden_state=<KerasTensor: shape=(None, 300, 768) dtype=float32 (created by layer 'tf_camembert_model_5')>, 
    #pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_camembert_model_5')>, 
    #hidden_states=None, attentions=None)

#Instanciation du modèle avec Keras
model_camemBERT = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = sortie_camemBERT, trainable=False)

#Définition du format d'entrée du modèle
entrees_ids = tf.keras.layers.Input(shape=(LONGUEUR_MAX_PARAGRAPH,), name='input_token', dtype='int32')
entrees_masks = tf.keras.layers.Input(shape=(LONGUEUR_MAX_PARAGRAPH,), name='masked_token', dtype='int32')
#?

#Création de la sortie du modèle
sortie_camemBERT = transformer_model([entrees_ids, entrees_masks])[0]
print(sortie_camemBERT.shape)
print(sortie_camemBERT)

l1 = Lambda(lambda seq: seq[:,0,:])(sortie_camemBERT) #pour ne récupérer que les vecteurs CLS
#print(l1.shape)

#lstm = LSTM(20, activation='tanh')(l1)
#dense1 = Dense(512, activation='relu')(l1)
#dense2 = Dense(256, activation='relu')(l1)
dense3 = Dense(128, activation='relu')(l1)
output = Dense(26, activation='softmax')(dense3)
print(output.shape)

model = tf.keras.Model(inputs=[entrees_ids, entrees_masks], outputs = output)
model.layers[2].trainable = False   #Désactive l'entraînement de camemBERT
print(model.layers[0])
print(model.layers[1])
print(model.layers[2])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

Some layers from the model checkpoint at jplu/tf-camembert-base were not used when initializing TFCamembertModel: ['lm_head']
- This IS expected if you are initializing TFCamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFCamembertModel were initialized from the model checkpoint at jplu/tf-camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


TFBaseModelOutputWithPooling(last_hidden_state=<KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_camembert_model_6')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_camembert_model_6')>, hidden_states=None, attentions=None)
(None, 512, 768)
KerasTensor(type_spec=TensorSpec(shape=(None, 512, 768), dtype=tf.float32, name=None), name='tf_camembert_model_6/roberta/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_camembert_model_6'")
(None, 26)
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 51

In [ ]:
history = model.fit([np.array(output_tokenizer_train['input_ids']), np.array(output_tokenizer_train['attention_mask'])],np.array(y_train),
                    epochs=4, batch_size=4, verbose=1, validation_data=([np.array(output_tokenizer_test['input_ids']), np.array(output_tokenizer_test['attention_mask'])],np.array(y_test)) 
          )

Epoch 1/2
3286/3286 [==============================] - 302s 90ms/step - loss: 2.1556 - accuracy: 0.3743 - val_loss: 1.6149 - val_accuracy: 0.5309
Epoch 2/2
 773/3286 [======>.......................] - ETA: 2:32 - loss: 1.5778 - accuracy: 0.5197

KeyboardInterrupt: ignored

In [ ]:
ajar1 = "Il n'a pas répondu. Il but un peu de thé de menthe qui est bon pour la santé. Monsieur machin portait toujours une machin grise, depuis quelque temps, pour ne pas être surpris en veston s'il était appelé. Il m'a regardé et a observé le silence. Il devait penser que j'étais encore interdit aux mineurs et qu'il y avait des choses que je ne devais pas savoir. En ce moment je devais avoir sept ans ou peut-être huit, je ne peux pas vous dire juste parce que je n'ai pas été daté, comme vous allez voir quand on se connaîtra mieux, si vous trouvez que ça vaut la peine"

In [ ]:
ajar2 = "Madame Rosa était née en Pologne comme Juive mais elle s'était défendue, au Maroc et en Algérie pendant plusieurs années et elle savait l'arabe comme vous et moi. Elle savait aussi le juif pour les mêmes raisons et on se parlait souvent dans cette langue. La plupart des autres locataires de l'immeuble étaient des Noirs. Il y a trois foyers noirs rue Bisson et deux autres où ils vivent par tribus, comme ils font ça en Afrique. Il y a surtout les Sarakollé, qui sont les plus nombreux et les Toucouleurs, qui sont pas mal non plus. Il y a beaucoup d'autres tribus rue Bisson mais je n'ai pas le temps de vous les nommer toutes. Le reste de la rue et du boulevard de Belleville est surtout juif et arabe. Ça continue Comme ça jusqu’à la Goutte d’Or et après c’est les quartiers français qui commencent."

In [ ]:
ajar3 = "Au début je ne savais pas que je n'avais pas de mère et je ne savais même pas qu'il en fallait une. Madame Rosa évitait d'en parler pour ne pas me donner des idées. Je ne sais pas pourquoi je suis né et qu'est-ce qui s'est passé exactement. Mon copain le Mahoute qui a plusieurs années de plus que moi m'a dit que c'est les conditions d'hygiène qui font ça. Lui était né à la Casbah à Alger et il était venu en France seulement après. Il n'y avait pas encore d'hygiène à la Casbah et il était né parce qu'il n'y avait ni bidet ni eau potable ni rien. Le Mahoute a appris tout cela plus tard, quand son père a cherché à se justifier et lui a juré qu'il n'y avait aucune mauvaise volonté chez personne. Le Mahoute m'a dit que les femmes qui se défendent ont maintenant une pilule pour l'hygiène mais qu'il était né trop tôt."

In [ ]:
ajar4 = "Il y avait chez nous pas mal de mères qui venaient une ou deux fois par semaine mais c'était toujours pour les autres. Nous étions presque tous des enfants de putes chez Madame Rosa, et quand elles partaient plusieurs mois en province pour se défendre là-bas, elles venaient voir leurs mômes avant et après. C'est comme ça que j'ai commencé à avoir des ennuis avec ma mère. Il me semblait que tout le monde en avait une sauf moi. "

In [ ]:
ajar5 = "J'ai commencé à avoir des crampes d'estomac et des convulsions pour la faire venir. Il y avait sur le trottoir d'en face un môme qui avait un ballon et qui m'avait dit que sa mère venait toujours quand il avait mal au ventre. J'ai eu mal au ventre mais ça n'a rien donné et ensuite j'ai eu des convulsions, pour rien aussi. J'ai même chié partout dans l'appartement pour plus de remarque. Rien. Ma mère n'est pas venue et Madame Rosa m'a traité de cul d'Arabe pour la première fois, car elle n'était pas française. "

In [ ]:
ajar6 = "Je lui hurlais que je voulais voir ma mère et pendant des semaines j'ai continué à chier partout pour me venger. Madame Rosa a fini par me dire que si je continuais c'était l'Assistance publique et là j'ai eu peur, parce que l'Assistance publique c'est la première chose qu'on apprend aux enfants. J'ai continué à chier pour le principe mais ce n'était pas une vie. On était alors sept enfants de putes en pension chez Madame Rosa et ils se sont tous mis à chier à qui mieux mieux car il n'y a rien de plus conformiste que les mômes et il y avait tant de caca partout que je passais inaperçu là-dedans."

In [ ]:
ajar_list = []

In [ ]:
for i in range(1,7,1):
  ajar_list.append(eval(f'ajar{i}'))

In [ ]:
ajar_token = tokenizer(ajar_list, max_length=LONGUEUR_MAX_PARAGRAPH, padding="max_length", truncation=True, return_tensors='tf', add_special_tokens=True)
inputs_ids = np.array(ajar_token['input_ids'])
attention_mask = np.array(ajar_token['attention_mask'])

In [ ]:
pred = model.predict([inputs_ids,attention_mask])

In [ ]:
le.inverse_transform([np.argmax(pred[i]) for i in range(0,6,1)])

array(['GARY Romain', 'MODIANO Patrick', 'DEBEAUVOIR Simone',
       'DURAS Marguerite', 'DURAS Marguerite', 'DURAS Marguerite'],
      dtype='<U22')

In [ ]:
print(pred[0])
le.classes_

[3.20807993e-02 3.59245198e-04 2.08586454e-04 1.91727374e-02
 4.24767099e-03 9.54917341e-05 6.39927760e-03 1.00717845e-03
 1.50003545e-02 1.79646508e-04 5.96367419e-01 6.47042086e-03
 9.18567032e-02 2.61141281e-07 1.37484303e-05 1.30430125e-02
 2.30684108e-03 1.98237047e-01 2.47214848e-05 1.37941664e-04
 4.40377480e-05 1.58131518e-03 2.21742131e-03 2.43368340e-04
 8.70327465e-03 1.37613176e-06]


array(['BASTIDE Francois-Regis', 'BAZIN Herve', 'BAZIN Hervé',
       'CAMUS Albert', 'CELINE Louis-Ferdinand', 'COHEN Albert',
       'DEBEAUVOIR SIMONE', 'DEBEAUVOIR Simone', 'DURAS Marguerite',
       'ECHENOZ Jean', 'GARY Romain', 'GASCAR Pierre', 'GIONO Jean',
       'GRAINVILLE Patrick', 'GUTH Paul', 'JOFFO Joseph', 'KESSEL Joseph',
       'MODIANO Patrick', 'PEREC Georges', 'QUEFFELEC Henri',
       'QUEFFELEC Yann', 'SAGAN Fraçoise', 'SARRAUTE Nathalie',
       'SCHOENDOERFFER Pierre', 'VIAN Boris', 'YOURCENAR Marguerite'],
      dtype='<U22')

In [ ]:
def dict_proba(pred):
  
  list_dicts = []
  authors = le.classes_ #label encoder

  for probas in pred:
    i = 0
    dict = {}
    for proba in probas:
      dict[authors[i]] = round(proba,2)
      i = i+1
    list_dicts.append(dict)
  
  return list_dicts

In [ ]:
dict_proba(pred)

[{'BASTIDE Francois-Regis': 0.03,
  'BAZIN Herve': 0.0,
  'BAZIN Hervé': 0.0,
  'CAMUS Albert': 0.02,
  'CELINE Louis-Ferdinand': 0.0,
  'COHEN Albert': 0.0,
  'DEBEAUVOIR SIMONE': 0.01,
  'DEBEAUVOIR Simone': 0.0,
  'DURAS Marguerite': 0.02,
  'ECHENOZ Jean': 0.0,
  'GARY Romain': 0.6,
  'GASCAR Pierre': 0.01,
  'GIONO Jean': 0.09,
  'GRAINVILLE Patrick': 0.0,
  'GUTH Paul': 0.0,
  'JOFFO Joseph': 0.01,
  'KESSEL Joseph': 0.0,
  'MODIANO Patrick': 0.2,
  'PEREC Georges': 0.0,
  'QUEFFELEC Henri': 0.0,
  'QUEFFELEC Yann': 0.0,
  'SAGAN Fraçoise': 0.0,
  'SARRAUTE Nathalie': 0.0,
  'SCHOENDOERFFER Pierre': 0.0,
  'VIAN Boris': 0.01,
  'YOURCENAR Marguerite': 0.0},
 {'BASTIDE Francois-Regis': 0.0,
  'BAZIN Herve': 0.0,
  'BAZIN Hervé': 0.0,
  'CAMUS Albert': 0.07,
  'CELINE Louis-Ferdinand': 0.0,
  'COHEN Albert': 0.0,
  'DEBEAUVOIR SIMONE': 0.0,
  'DEBEAUVOIR Simone': 0.0,
  'DURAS Marguerite': 0.16,
  'ECHENOZ Jean': 0.0,
  'GARY Romain': 0.23,
  'GASCAR Pierre': 0.0,
  'GIONO Jean'